### **Phase 2: Ball Tracking and Player Statistics with Bytetrack** 🎥⚽


1. **Player & Ball Tracking** ⚽

In [3]:
# test with cli command
!yolo track model="/Users/alyazouzou/Desktop/CV_Football/FootCVision/phase1/runs/detect/train/weights/best.pt" source="/Users/alyazouzou/Desktop/CV_Football/vids/good.mov" conf=0.3 iou=0.5 show

Ultralytics 8.3.57 🚀 Python-3.9.21 torch-2.5.1 CPU (Apple M3)
YOLO11s summary (fused): 238 layers, 9,414,348 parameters, 0 gradients, 21.3 GFLOPs

video 1/1 (frame 1/4159) /Users/alyazouzou/Desktop/CV_Football/vids/good.mov: 384x640 1 ball, 8 players, 61.7ms
2025-03-16 14:31:36.781 python[6846:372344] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-16 14:31:36.781 python[6846:372344] +[IMKInputSession subclass]: chose IMKInputSession_Modern
video 1/1 (frame 2/4159) /Users/alyazouzou/Desktop/CV_Football/vids/good.mov: 384x640 1 ball, 7 players, 59.7ms
video 1/1 (frame 3/4159) /Users/alyazouzou/Desktop/CV_Football/vids/good.mov: 384x640 1 ball, 8 players, 51.0ms
video 1/1 (frame 4/4159) /Users/alyazouzou/Desktop/CV_Football/vids/good.mov: 384x640 1 ball, 8 players, 56.2ms
video 1/1 (frame 5/4159) /Users/alyazouzou/Desktop/CV_Football/vids/good.mov: 384x640 1 ball, 8 players, 55.6ms
video 1/1 (frame 6/4159) /Users/alyazouzou/Desktop/CV_Football/vids/good.mov: 384x640 1 ball, 8 playe

#### **With KmeansClassifier** 🧠

In [1]:
import cv2
from trackkmeans import TrackKMeans

In [2]:
video_path = "/Users/alyazouzou/Desktop/CV_Football/FootCVision/utils/vid/example1.mov"
tracker = TrackKMeans(video_path)
tracker.track_and_classify(save_video=True)

Extracting embeddings: 0it [00:00, ?it/s]

Processing frames: 194it [06:47,  2.10s/it]

🔍 Detected 15 players, 0 ball(s)
⚠️ No players or ball detected. Skipping possession update.

0: 352x640 15 players, 2 referees, 219.1ms
Speed: 15.4ms preprocess, 219.1ms inference, 0.4ms postprocess per image at shape (1, 3, 352, 640)


Extracting embeddings: 0it [00:00, ?it/s]

Processing frames: 195it [06:55,  4.07s/it]

...

Extracting embeddings: 0it [00:00, ?it/s]

Processing frames: 209it [07:23,  1.94s/it]

🔍 Detected 17 players, 0 ball(s)
⚠️ No players or ball detected. Skipping possession update.

0: 352x640 18 players, 1 referee, 193.0ms
Speed: 2.4ms preprocess, 193.0ms inference, 0.4ms postprocess per image at shape (1, 3, 352, 640)


Process Process-auto_conversion:
Process Process-auto_conversion:
Processing frames: 209it [07:24,  2.12s/it]
Traceback (most recent call last):
  File "/Users/alyazouzou/miniconda3/envs/footcvision/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/alyazouzou/miniconda3/envs/footcvision/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/alyazouzou/miniconda3/envs/footcvision/lib/python3.9/site-packages/transformers/safetensors_conversion.py", line 84, in auto_conversion
    sha = get_conversion_pr_reference(api, pretrained_model_name_or_path, **cached_file_kwargs)
  File "/Users/alyazouzou/miniconda3/envs/footcvision/lib/python3.9/site-packages/transformers/safetensors_conversion.py", line 71, in get_conversion_pr_reference
    spawn_conversion(token, private, model_id)
  File "/Users/alyazouzou/miniconda3/envs/footcvision/lib/python3.9/site-packages/transformers/safeten

KeyboardInterrupt: 

![Frame Example](../utils/img/kmeans_track_ex.png)

#### **With HSVClassifier** 🎥

In [1]:
import cv2
from trackhsv import TrackHSV

In [ ]:
video_path = "/Users/alyazouzou/Desktop/CV_Football/vids/good.mov"
tracker = TrackHSV(video_path)
tracker.track_and_classify_hsv(save_video=False)

![Frame Example](../utils/img/hsv_track_ex.png)

2. **Statistics** 📈
   - Extract metrics for each player, such as:
     - Distance covered.
     - Ball possession time.
     - Speed and acceleration.